<a href="https://colab.research.google.com/github/oviyajay/ML-Final-Project/blob/main/MLfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
import os 
import cv2
!pip install tflearn
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 

#Members:...
#https://blog.paperspace.com/image-captioning-with-ai/
#image preprocessing

TEST_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/with_mask"
TEST_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/without_mask"
TRAIN_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/with_mask"
TRAIN_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/without_mask"

!git clone https://github.com/oviyajay/ML-Final-Project

#label the images
#create training data and test data 


     |████████████████████████████████| 112kB 73kB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=be8d4a7911561493d81a3db8a99920d23011e45f9074421ed7a1f2ac7ac341c0
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn
Instructions for updating:
non-resource variables are not supported in the long term
Cloning into 'ML-Final-Project'...
remote: Enumerating objects: 889, done.
remote: Counting objects: 100% (889/889), done.
remote: Compressing objects: 100% (875/875), done.
remote: Total 889 (delta 20), reused 826 (delta 0), pack-reused 0
Receiving objects: 100% (889/889), 25.50 MiB | 25.20 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [5]:
#Eric 
train_path = "/content/ML-Final-Project/maskdata/maskdata/train/"
os.listdir(train_path)

test_path = "/content/ML-Final-Project/maskdata/maskdata/test/"
os.listdir(test_path)



['with_mask', 'without_mask']

In [13]:
label_names = ['with_mask', 'without_mask']
IMG_SIZE = 256

def load_data(path):
  data = []

  for label in label_names:
    for filename in os.listdir(os.path.join(path, label)):
      img_file = os.path.join(path, label, filename)
      # loading the image from the path and then converting them into 
      # greyscale for easier covnet prob 
      img = cv2.imread(img_file, cv2.IMREAD_COLOR) # Grayscale => number_of_channels=1, RGB => number_of_channels=3

      # resizing the image for processing them in the covnet 
      img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

      label_id = label_names.index(label)

      # final step-forming the training data list with numpy array of the images 
      data.append([np.array(img), np.array(label_id)]) 




In [14]:
train_data = load_data(train_path)
test_data = load_data(test_path)


In [18]:
# train is a list of [x, y] pairs
train_x = [i[0] for i in train_data]  #i[0] is a numpy array that is the size/shape of a single image
train_x = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE)  # num_image, width, height, channels
train_y = [i[1] for i in train_data]
test_x = [i[0] for i in test_data]
test_x = np.array(test_x).reshape(-1, IMG_SIZE, IMG_SIZE) 
test_y = [i[1] for i in test_data]

TypeError: ignored

In [ ]:
#Iant

In [ ]:
#Anindita

In [ ]:
#Oviya

